## Import necessary libraries

In [40]:
import ee
import geemap
import os
import datetime as dt
from datetime import datetime, timedelta
import numpy as np

In [3]:
#!pip install ffmpeg-python

In [2]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Enter region of interest and date

In [4]:
# Mukund's ROI
hs = "Arctic"

if hs == "Arctic":
    bbox = [-12, 75, 6, 79]
    start_date = "2023-04-01"
    end_date = "2023-05-31"
    crs = "EPSG:32631" #"EPSG:3857" 
    dim = 500
    orbits = []
    # Approximately (72.6 to 79.2 N) and (-10 to 12 E) April to May 2023
elif hs == "Antarctic":
    bbox = [-176.3, -76.3, -153.9, -73.5]
    start_date = "2023-11-01"
    end_date = "2023-12-31"
    crs = "EPSG:32703" #"EPSG:3857" #
    dim = 700
    orbits = [] # [111, 97, 112, 140, 141, 9, 38, 39, 67] #170, 120, 171, 24
    # Approximately (-76.3 to -74.4 S) and (-176.3 to -153.9 W) November - December 2023

region = ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]) #ee.Geometry.Rectangle(bbox)
point = ee.Geometry.Point([np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]).buffer(20000)

print(bbox)

[-12, 75, 6, 79]


## Generate GIF time series

In [5]:
collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)

In [6]:
img_names = collection.aggregate_array('system:id').getInfo()

for name in img_names:
    img = ee.Image(name)
    bands = img.bandNames().getInfo()
    # print(bands)

In [89]:
img2 = collection_addbands(img)

In [ ]:
band = img.bandNames() # First band ('HH' or 'VV')
band = band.contains('HH').get()
norm = img.select(band).divide(img.select('angle')).rename('norm')
img2 = img.addBands(norm, overwrite=True)

In [78]:
pi = ee.Number.expression('Math.PI', None)
sina = img.select('angle').multiply(pi).divide(180).sin()
cosa = img.select('angle').multiply(pi).divide(180).cos()

In [91]:
Map = geemap.Map()
Map.addLayer(img.select('VV'), {'min':-30, 'max':0}, 'VV')
Map.addLayer(img.select('VV').divide(sina), {'min':-30, 'max':0}, 'VV_cos')
Map.addLayer(img2, {'min':-1, 'max':0}, 'VV2')
Map.addLayer(sina, {'min':0, 'max':1}, 'sina')
Map.addLayer(sina, {'min':0, 'max':1}, 'cosa')
Map.addLayer(img.select('angle'), {'min':0, 'max':90}, 'a')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [36]:
bands = img.bandNames()
bands == ee.List(['VV', 'VH', 'angle'])

False

In [32]:
ee.Algorithms.If(band.contains('HH'), 'HH', 'VV')

In [10]:
collection0 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)
collection1 = collection0.map(collection_addbands)
collection = collection1.map(add_coverage)

In [41]:
collection.first().select("norm").projection().crs().getInfo()

'EPSG:32631'

In [49]:
crs

'EPSG:32631'

In [6]:
def collection_addbands(img):
    bands = img.bandNames() # First band ('HH' or 'VV')
    band = [ee.Algorithms.If(bands.contains('HH'), 'HH', 'VV')]
    norm = img.select(band).divide(img.select('angle')).rename('norm')
    img2 = img.addBands(norm, overwrite=True).select('norm')
    return img2

def add_coverage(img):
    
    tol = 10000
    overlap = img.geometry().intersection(region, tol)
    ratio = overlap.area(tol).divide(region.area(tol))
    return img.set({'coverage_ratio': ratio})

# calculate coverage area of image to roi
def coverage(img):
    tol = 10000
    overlap = img.geometry().intersection(region, tol)
    ratio = overlap.area(tol).divide(region.area(tol))
    return ratio.getInfo()

#### Regular date range

In [5]:
for year in [2018, 2019, 2020, 2021, 2022, 2023]:
    step = 3
    for month in range(1, 13, step):
        start_date = f"{year}-{str(month).zfill(2)}-01"
        if month < 12-step:
            end_date = f"{year}-{str(month+step).zfill(2)}-01"
        else:
            end_date = f"{year+1}-01-01"

        # start_date = "2023-11-01"
        # end_date = "2023-12-31"
        
        # collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)\
        # .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH')).select('HH')
        
        collection0 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)
        collection1 = collection0.map(collection_addbands)
        collection = collection1.map(add_coverage).filter(ee.Filter.gt('coverage_ratio', 0.1))

        # crs = collection.first().select("norm").projection().crs().getInfo()

        # Define arguments for animation function parameters.
        video_args = {
            "dimensions": dim,
            "region": region,
            "framesPerSecond": 1.5,
            "bands": ["norm"],
            "crs":crs, #
            "min": -0.8,
            "max": -0.2,
            "gamma": 1    
        }
        
        if len(orbits) > 0:
            collection = collection.filter(ee.Filter.inList('relativeOrbitNumber_start', orbits))
        # .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 34),
        #         ee.Filter.eq('relativeOrbitNumber_start', 136)))
        
        images = geemap.create_timeseries(
            collection, start_date, end_date, region, frequency="day", reducer="median"
        )
        
        s = images.aggregate_array('system:time_start').getInfo()
        
        days = []
        for sec in s:
            d = dt.datetime(1970, 1, 1) + dt.timedelta(seconds = sec / 1000)
            days.append(d.strftime("%Y-%m-%d"))
        
        print(start_date, end_date, images.size().getInfo())
        
        work_dir = "D:\\Floes\\S1"
        
        out_gif = os.path.join(work_dir, f"S1_W{bbox[0]}S{bbox[1]}_{start_date}_{end_date}.gif")
        geemap.download_ee_video(images, video_args, out_gif)
        
        texted_gif = out_gif.replace(".gif", "_text.gif") #os.path.join(work_dir, f"S1_{hs}_{start_date}_{end_date}_text.gif")
        geemap.add_text_to_gif(
            out_gif,
            texted_gif,
            xy=("3%", "3%"),
            text_sequence=days,
            font_size=20,
            duration=800,
            font_color="#ffffff",
            add_progress_bar=False,
        )

        if os.path.exists(out_gif):
            os.remove(out_gif)
        
        print("====================================================")

print("DONE!")

2018-01-01 2018-04-01 77
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_W-176.3S-76.3_2018-01-01_2018-04-01.gif
2018-04-01 2018-07-01 59
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_W-176.3S-76.3_2018-04-01_2018-07-01.gif
2018-07-01 2018-10-01 65
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_W-176.3S-76.3_2018-07-01_2018-10-01.gif
2018-10-01 2019-01-01 74
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_W-176.3S-76.3_2018-10-01_2019-01-01.gif
2019-01-01 2019-04-01 73
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_W-176.3S-76.3_2019-01-01_2019-04-01.gif
2019-04-01 2019-07-01 52
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_W-176.3S-76.3_2019-04-01_2019-07-01.gif
2019-07-01 2019-10-01 59
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_W-176.3S-76

#### Specific start and end dates

In [10]:
start_date = "2020-11-01"
end_date = "2020-11-10"

# collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)\
# .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH')).select('HH')

collection0 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)
collection1 = collection0.map(collection_addbands)
collection = collection1.map(add_coverage).filter(ee.Filter.gt('coverage_ratio', 0.1))

In [23]:
img = collection.first() #.clip(region)
img

In [13]:
from PIL import Image
im = Image.open('C:\\Users\\yoko2261\\AppData\\Local\\Temp\\d53802a8-fd38-4c63-a5bd-1adc3fddc36c.gif')

In [16]:
import imageio
im_array = imageio.imread('C:\\Users\\yoko2261\\AppData\\Local\\Temp\\d53802a8-fd38-4c63-a5bd-1adc3fddc36c.gif')

In [25]:
images = geemap.create_timeseries(
    collection, start_date, end_date, region, frequency="day", reducer="median", parallel_scale=2
)


In [26]:
img = images.first()

In [27]:
img

In [ ]:


# crs = collection.first().select("norm").projection().crs().getInfo()

# Define arguments for animation function parameters.
video_args = {
    "dimensions": dim,
    "region": region,
    "framesPerSecond": 1.5,
    "bands": ["norm"],
    "crs":crs, #
    "min": -0.8,
    "max": -0.2,
    "gamma": 1    
}

if len(orbits) > 0:
    collection = collection.filter(ee.Filter.inList('relativeOrbitNumber_start', orbits))
# .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 34),
#         ee.Filter.eq('relativeOrbitNumber_start', 136)))

images = geemap.create_timeseries(
    collection, start_date, end_date, region, frequency="day", reducer="median", parallel_scale=2
)

s = images.aggregate_array('system:time_start').getInfo()

days = []
for sec in s:
    d = dt.datetime(1970, 1, 1) + dt.timedelta(seconds = sec / 1000)
    days.append(d.strftime("%Y-%m-%d"))

print(start_date, end_date, images.size().getInfo())

work_dir = "D:\\Floes\\S1"

out_gif = os.path.join(work_dir, f"S1_W{bbox[0]}S{bbox[1]}_{start_date}_{end_date}.gif")
geemap.download_ee_video(images, video_args, out_gif)

texted_gif = out_gif.replace(".gif", "_text.gif") #os.path.join(work_dir, f"S1_{hs}_{start_date}_{end_date}_text.gif")
geemap.add_text_to_gif(
    out_gif,
    texted_gif,
    xy=("3%", "3%"),
    text_sequence=days,
    font_size=20,
    duration=800,
    font_color="#ffffff",
    add_progress_bar=False,
)

if os.path.exists(out_gif):
    os.remove(out_gif)

print("====================================================")

print("DONE!")

In [ ]:
create_timelapse(collection, start_date, end_date, region=None, bands=None, frequency='year', reducer='median', date_format=None, out_gif=None, palette=None, vis_params=None, dimensions=768, frames_per_second=10, crs='EPSG:3857'

### Individual images download in original resolutions

In [32]:
start_date = "2018-10-02"
end_date = "2018-10-20"

# collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)\
# .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH')).select('HH')


collection0 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)
collection1 = collection0.map(collection_addbands)
collection = collection1.map(add_coverage).filter(ee.Filter.gt('coverage_ratio', 0.5))

S1_ids = collection.aggregate_array('system:id').getInfo()

In [33]:
collection

In [86]:
# Mukund's ROI
hs = "Arctic"

if hs == "Arctic":
    bbox = [-13, 75.5, -2, 78]
    start_date = "2023-04-01"
    end_date = "2023-05-31"
    crs = "EPSG:32631" #"EPSG:3857" 
    dim = 500
    orbits = []
    # Approximately (72.6 to 79.2 N) and (-10 to 12 E) April to May 2023
elif hs == "Antarctic":
    bbox = [-176.3, -76.3, -153.9, -73.5]
    start_date = "2023-11-01"
    end_date = "2023-12-31"
    crs = "EPSG:32703" #"EPSG:3857" #
    dim = 700
    orbits = [] # [111, 97, 112, 140, 141, 9, 38, 39, 67] #170, 120, 171, 24
    # Approximately (-76.3 to -74.4 S) and (-176.3 to -153.9 W) November - December 2023

region = ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]) #ee.Geometry.Rectangle(bbox)
point = ee.Geometry.Point([np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]).buffer(20000)

print(bbox)

[-13, 75.5, -2, 78]


In [99]:
rs = [0.56, 0.98]

In [105]:
img

In [115]:
os.path.basename(i)[17:32]

'20181009T073813'

In [121]:
start_date = "2018-10-02" #02
end_date = "2018-10-20" #20

d0 = datetime.strptime(start_date, "%Y-%m-%d")
d2 = datetime.strptime(end_date, "%Y-%m-%d")

dn = abs(d2-d0).days

map = geemap.Map()

for i in range(dn):

    t1 = (d0 + timedelta(days = i))
    t2 = (d0 + timedelta(days = i+1))

    t1_str = t1.strftime("%Y-%m-%d")
    
    collection0 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(t1, t2)\
    .filter(ee.Filter.eq('instrumentMode', 'EW'))
    
    collection1 = collection0.map(collection_addbands)
    collection = collection1.map(add_coverage).filter(ee.Filter.gt('coverage_ratio', 0.7))

    S1_ids = collection.aggregate_array('system:id').getInfo()
        
    # print(t1, t2, len(S1_ids))

    # print(start_date, end_date, len(S1_ids))

    if len(S1_ids) > 1:
        rs = collection.aggregate_array('coverage_ratio').getInfo()
        S1_ids = [S1_ids[np.argmax(rs)]]

    for i in S1_ids:
        
        img = collection_addbands(ee.Image(i)).clip(region)
        img = add_coverage(img)
        r = img.get('coverage_ratio').getInfo()
        # img = collection.reduce(ee.Reducer.median()).select('norm_median').clip(region)

        name = os.path.basename(i)[17:32]
        description = f"S1_{name}"
    
        print(description, r)
        task = ee.batch.Export.image.toDrive(
            image=img,
            description=description,
            folder="Eddies",
            scale=40,
            maxPixels = 1e12,
            crs=crs, #projection['crs'],
            # crsTransform=projection['transform'],
            region=region,
        )
        task.start()
    
        map.addLayer(img, {'max': 0, 'min':-0.8}, os.path.basename(i))

S1_20181002T074615 0.9966595379614944
S1_20181003T073726 0.9858874395615144
S1_20181004T072958 0.9294351728085096
S1_20181005T072054 0.7745495556699334
S1_20181006T080155 0.8434128685128911
S1_20181007T075439 0.8188496150020085
S1_20181008T074529 0.9966501297765696
S1_20181009T073813 0.9392561588500072
S1_20181010T072915 0.9408609559759561
S1_20181011T072144 0.773109720523525
S1_20181012T080235 0.8436876974560304
S1_20181013T075346 0.9495062333240208
S1_20181014T074615 0.9966603042571968
S1_20181015T073726 0.9858855519128003
S1_20181016T072958 0.9294230100325082
S1_20181017T072054 0.774269980493956
S1_20181018T080155 0.8438386941800144
S1_20181019T075439 0.8190262741880638


In [119]:
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…